# Validation of the different aspects of the model

Four aspects of the model are compared to the observational data: events detection, density of the layers (through the SWE values), height of the layers and temperature profile.

## 0. Simulated snowpack creation

In [1]:
# Imports

%run little_awk_functions.py
%run parameters.py

In [2]:
# Clean dataset

data_set_used = xr.open_dataset('snow_pit_1_filled.nc')

data_set_used = data_set_used.ffill(dim='time')

median_space_filtering(data_set_used, 5, x_span=7)
median_time_filtering(data_set_used, 11)

data_set_used['snow_surface'] = data_set_used['snow_surface'] - data_set_used['snow_surface'].isel(x=x_sel, y=y_sel).dropna('time').min()

---> Median filtering in space with a window [7, 11]
---> Median filtering in time with a window of 11


In [3]:
# TODO refaire les cells suivantes for each point and store end data for stats?

In [4]:
# Define dates

data_starting_date_in_ns = float(data_set_used.time.values[0])

data_starting_date_in_s = pd.to_datetime(data_set_used.time.values[0]).timestamp()
data_ending_date_in_s = pd.to_datetime(data_set_used.time.values[-1]).timestamp()
data_duration_in_s = data_ending_date_in_s - data_starting_date_in_s
nb_iterations = int(data_duration_in_s/dt + 1)

In [5]:
# Get timing of events

results = get_snow_events(data_set_used, x_sel, y_sel, time_window_std, std_threshold)
start_accumulation, start_erosion, end_accumulation, end_erosion = results[0], results[1], results[2], results[3]

# Convert end times into more manageable orders of magnitude (seconds)

end_accumulation_times = data_set_used.snow_surface.isel(x=x_sel, y=y_sel, time=end_accumulation)
end_accumulation_times = (pd.to_datetime(end_accumulation_times.time).astype(int) - data_starting_date_in_ns) / 1000000000  # in s

end_erosion_times = data_set_used.snow_surface.isel(x=x_sel, y=y_sel, time=end_erosion)
end_erosion_times = (pd.to_datetime(end_erosion_times.time).astype(int) - data_starting_date_in_ns) / 1000000000  # in s


In [6]:
# Simulate the snowpack

snowpack = simulate_snowpack_evolution(data_set_used, x_sel, y_sel, nb_iterations, end_accumulation_times, end_erosion_times,
                                       start_accumulation, end_accumulation, start_erosion, end_erosion, jj, dt, ro_layer, ro_water, ro_ice,
                                       t_old, tf, tsfc, dy_snow, age_layers, gamma, cp_snow, melt_flag, a1, a2, met_temp_data=met_temp,
                                       met_time_data=met_time, new_snow_ro=simul_new_snow_ro,
                                       fit_top_of_snowfall_to_curve=simul_fit_top_of_snowfall_to_curve)

ro_layer_evolution, depth_evolution, temperature_evolution = snowpack[0], snowpack[1], snowpack[2]


## Useful functions for what follows

In [18]:
def get_data_from_marker(index_of_marker_layer, ro_layer_array, depth_array):
    '''
    Function that computes some characteristic values (SWE, height, average density) above a "marker" interface
    Args:
        index_of_marker_layer: index of the layer that is directly underneath the marker interface
        ro_layer_array: array of the densities of each layers at the end of the simulation
        depth_array: array of the heights of each layers at the end of the simulation
    Returns:
        swe_from_marker: value of the SWE above the given interface
        height_from_marker: height of the snowpack above the given interface
        ave_density_from_marker: averaged density of the snow above the given interface
    '''
    swe_from_marker = np.dot(np.array(ro_layer_array[index_of_marker_layer+1:]), np.array(depth_array[index_of_marker_layer+1:])) / 1000
    height_from_marker = sum(depth_array[i] for i in range(index_of_marker_layer+1, len(depth_array)))
    ave_density_from_marker = np.dot(np.array(ro_layer_array[index_of_marker_layer+1:]), np.array(depth_array[index_of_marker_layer+1:])) / height_from_marker
    
    return(swe_from_marker, height_from_marker, ave_density_from_marker)

## 1. Events detection validation

This will be done by eye simply, most probably.

In [7]:
# - create a measure to compare observation/simulation: number of events that match manually detected events / total nb of detected events (?)
#      or use the 3 measures used for sensitivity tests?
# - make a simulation for each pit point (in this case, get events timing)
# - get observational data for each pit point (in this case, manually detect events)
# - make a graph to present results (in this case: ??) // fig 5 with ages of layers vs observations?

## 2. Density and SWE validation

In [24]:
# TODO:
# - make a density profile: > function(lowest depth to take into account, frequency of sampling)
#     - get frequency at which density was sampled
#     - make an array of the layer index at each sampled depth > reusable in other functions > make independant and give as input
#     - make an array of the density at each of these depths, starting from a given depth
# - get the density profile from the observations > make a netcdf file from all the pits? > can use the previous function?
# - compute the correlation between them: simply re-use the previously coded correlation function
# - plot both profiles, with depth on the y-axis going downwards, and density on the x-axis
# - do this for several pits and save the results to do statistics

In [ ]:
# TODO:
# - get all marker layers' indices
# - compute all SWEs above those layers in observational data (netcdf?)
# - compute all SWEs above those layers in simulations
# - save results for statistics

## 3. Height of layers validation

In [ ]:
# TODO:
# - get all marker layers' indices
# - compute all heights above those layers in observational data (netcdf?)
# - compute all heights above those layers in simulations
# - repeat the two previous steps with total height of snowpack
# - save results for statistics

## 4. Temperature profile validation

In [25]:
# TODO:
# - make a temperature profile: > function(lowest depth to take into account, frequency of sampling)
#     - get frequency at which temperature was sampled
#     - make an array of the layer index at each sampled depth > reusable in other functions
#     - make an array of the temperature at each of these depths, starting from a given depth
# - get the temperature profile from the observations > make a netcdf file from all the pits? > can use the previous function?
# - compute the correlation between them: simply re-use the previously coded correlation function
# - plot both profiles, with depth on the y-axis going downwards, and temp on the x-axis
# - do this for several pits and save the results to do statistics